# Telegram BOT
## In this Notebook we aim to develop a simple I/O bot from Telegram


![logo](https://raw.githubusercontent.com/python-telegram-bot/logos/master/logo-text/png/ptb-logo-text_768.png)

We are using https://github.com/python-telegram-bot/python-telegram-bot api

Introduction to the library https://github.com/python-telegram-bot/python-telegram-bot/wiki/Introduction-to-the-API

In [321]:
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [322]:
import telegram
token_telegram = '666164429:AAFPkMnbqRKXqCPvv-OtycJ5K5y4RQ3k_U4'
bot = telegram.Bot(token=token_telegram)

In [323]:
print(bot.get_me())

{'id': 666164429, 'first_name': 'SCIBOT', 'is_bot': True, 'username': 'sci_bot'}


In [324]:
updates = bot.get_updates()

In [325]:
print([u.message.text for u in updates])

[]


In [13]:
chat_id

191481115

In [16]:
bot.send_message(chat_id,"How about this?")

In [17]:
updates = bot.get_updates()
print([u.message.text for u in updates])

['/start', 'dd']


In [18]:
bot.send_photo(chat_id=chat_id, photo='https://raw.githubusercontent.com/python-telegram-bot/logos/master/logo-text/png/ptb-logo-text_768.png')

In [20]:
bot.send_document(chat_id, document=open('07154457.pdf','rb'))

## Mensage Handler

In [3]:
from telegram.ext import CommandHandler
from telegram.ext import Updater
bot_token = '666164429:AAFPkMnbqRKXqCPvv-OtycJ5K5y4RQ3k_U4'
updater = Updater(token=bot_token)

In [ ]:
dispatcher =  updater.dispatcher

#### Command Handler

In [15]:
from  subprocess import call

In [18]:
def blow_world(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text="BOOM!")
def turnoff(bot,update):
    print("TURNING OFF")
    call("shutdown now -P",shell=True)

In [9]:
blow_world_handler = CommandHandler('world', blow_world)
dispatcher.add_handler(blow_world_handler)

In [19]:
off_handler = CommandHandler('turnoff', turnoff)
dispatcher.add_handler(off_handler)

In [20]:
updater.start_polling() # Set It into the pool

TURNING OFF


In [6]:
from telegram.ext import MessageHandler, Filters
blow_world_handler = MessageHandler(Filters.text, blow_world)
dispatcher.add_handler(blow_world_handler)

In [7]:
dispatcher.handlers

{0: [<telegram.ext.messagehandler.MessageHandler at 0x7faae42cc2e8>]}

In [35]:
blow_world_handler

# MQTT BOT

### Publisher

In [27]:
import paho.mqtt.client as mqtt
import json
user = "j9j976fenpqs"
password = "IjzR8T18LJty"
publishAddress = "data/j9j976fenpqs/pub/out"


# user = "ou83rio3h43j"
# password = "anioaDro9R87"
# publishAddress = "data/ou83rio3h43j/pub/out"

client = mqtt.Client()
client.username_pw_set(user, password)
client.connect("mqtt.demo.konkerlabs.net", 1883)
client.publish(publishAddress, 
                 json.dumps({"temperature": 225, "unit": "celsius"}))

### Subscriber

In [ ]:
import paho.mqtt.client as mqtt
import json

user = "ou83rio3h43j"
password = "anioaDro9R87"
def on_connect(client, userdata, flags, rc):
    print("Connected!")
    client.subscribe("data/ou83rio3h43j/sub/in")

def on_message(client, data, msg):
    print(msg.topic + " " + str(msg.payload))


client = mqtt.Client()
client.on_message = on_message
client.on_connect = on_connect
client.username_pw_set(user, password)
client.connect("mqtt.demo.konkerlabs.net", 1883)
client.loop_forever()

Connected!
data/ou83rio3h43j/sub/in b'{"temperature": 225, "unit": "celsius"}'
data/ou83rio3h43j/sub/in b'{"temperature": "BOOM!", "unit": "celsius"}'


# Scrapy

In [2]:
import pandas as pd
tables = pd.read_html("https://www.prefeitura.unicamp.br/apps/site/cardapio.php")

In [326]:
cardapio = tables[1].loc[2:].copy()
cardapio_header = cardapio.iloc[0]
cardapio_header.name = ''
cardapio = pd.DataFrame(cardapio.values[1:], columns=cardapio_header)
cafe = tables[1].loc[0:1].copy()
cafe = cafe.drop(cafe.columns[1:],axis=1)
cafe_header =cafe.iloc[0]
cafe = cafe.iloc[1]
cafe_header.name = ''
cafe = pd.DataFrame(cafe.values, columns=cafe_header)
cardapio = pd.concat([cardapio, cafe],axis=1)

# REFEIÇÕES
almoco = cardapio["Almoço"].values[0].upper()
almoco_veg = cardapio["Almoço Vegetariano"].values[0].upper()
jantar = cardapio["Jantar"].values[0].upper()
jantar_veg = cardapio["Jantar Vegetariano"].values[0].upper()
cafe = cardapio["Café da manhã"].values[0].upper()

In [313]:
guarnicao = "GUARNIÇÃO"
prato_principal = "PRATO PRINCIPAL:"
salada = "SALADA:"
sobremesa = "SOBREMESA:"
suco = "SUCO:"
obs = "OBSERVAÇÕES:"
def extrai_refeicao(ref,ref_op):
    if ref_op == guarnicao and prato_principal in ref :
        return ref[0:ref.find(prato_principal)].strip()
    if ref_op == prato_principal and prato_principal in ref and salada in ref:
        pp = ref.find(prato_principal) + len(prato_principal)
        return ref[pp:ref.find(salada)].strip()
    elif ref_op == salada and salada in ref and sobremesa in ref:
        sp = ref.find(salada) + len(salada)
        return ref[sp:ref.find(sobremesa)].strip()
    elif ref_op == sobremesa and sobremesa in ref and suco in ref:
        sp = ref.find(sobremesa) + len(sobremesa)
        return ref[sp:ref.find(suco)].strip()
    elif ref_op == suco and suco in ref and obs in ref:
        sp = ref.find(suco) + len(suco)
        return ref[sp:ref.find(obs)].strip()
    elif ref_op == obs and obs in ref:
        op = ref.find(obs) + len(obs)
        return ref[op:].strip()
    else: return ""